## Explore

In [13]:
filename='9x9_10games'
#filename='gogod_9x9_games'

### TEST

In [14]:
import gzip
import struct
    
with gzip.open(filename+'-test-labels-idx1-ubyte.gz') as bytestream:
    magic_number=bytestream.read(4)
    
with gzip.open(filename+'-test-vectors-idx4-ubyte.gz') as bytestream:
    magic_number=bytestream.read(4)
    num_images=struct.unpack('>I',bytestream.read(4))[0]
    size_1=bytestream.read(4)
    size_2=bytestream.read(4)
    size_3=bytestream.read(4)
print(num_images,size_1,size_2,size_3)

(76, '\x00\x00\x00\x04', '\x00\x00\x00\t', '\x00\x00\x00\t')


### TRAIN

In [15]:
import gzip
import struct
    
with gzip.open(filename+'-train-labels-idx1-ubyte.gz') as bytestream:
    magic_number=bytestream.read(4)

with gzip.open(filename+'-train-vectors-idx4-ubyte.gz') as bytestream:
    magic_number=bytestream.read(4)
    num_images=struct.unpack('>I',bytestream.read(4))[0]
    size_1=bytestream.read(4)
    size_2=bytestream.read(4)
    size_3=bytestream.read(4)
print(num_images,size_1,size_2,size_3)

(430, '\x00\x00\x00\x04', '\x00\x00\x00\t', '\x00\x00\x00\t')
